# Agency Prefix Code Profiles
Which agencies apply the most or have the most prefix codes? Which ones have the least? 

Data: 
* [Oblgiated list](https://dot.ca.gov/programs/local-assistance/reports/e-76-obligated)

* running notebook with updated locode data 'obligated_list_1202.csv'

In [1]:
import pandas as pd
from siuba import *

import numpy as np
from calitp import *

import ipywidgets as widgets
from ipywidgets import *
from IPython.display import Markdown
from IPython.core.display import display

In [2]:
df = pd.read_csv('gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/obligated_list_1202.csv', low_memory=False)


/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:73: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [3]:
df.head()

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,warning,projectID,projectNO,compare_id_locode
0,Obligated,BPMPL,5904(121),Humboldt County,2018-12-18,2018-12-18,2018-12-18,2018-12-18,2018-12-27,0.00,...,14 Bridges In Humboldt County,Bridge Preventive Maintenance - Deck Joints,3,NaN,NaN,NON-MPO,NaN,5904,121,True
1,Obligated,ER,32D0(008),Mendocino County,2018-12-17,2018-12-19,2018-12-20,2018-12-20,2018-12-27,11508.00,...,"Comptche Ukiah Road, Cr 223 Pm 17.25",Permanent Restoration,3,2018-12-17,2018-12-18,NON-MPO,NaN,32D0,8,False
2,Obligated,ER,4820(004),Humboldt County,2018-12-07,2018-12-21,2018-12-21,2018-12-21,2018-12-27,45499.64,...,Mattole Rd Pm 43.17,Permanent Restoration,5,2018-12-06,2018-12-07,NON-MPO,NaN,4820,4,False
3,Obligated,CML,5924(244),Sacramento County,2018-12-11,2018-12-11,2018-12-21,2018-12-27,2018-12-27,207002.00,...,Fair Oaks Blvd. Between Howe Ave And Munroe St,Create A Smart Growth Corridor With Barrier Se...,1,2018-12-07,2018-12-07,SACOG,NaN,5924,244,True
4,Obligated,CML,5924(214),Sacramento County,2018-12-05,2018-12-11,2018-12-21,2018-12-27,2018-12-27,0.00,...,Florin Rd Between Power Inn Rd. And Florin Per...,Streetscape (tc),3,2018-11-28,2018-12-04,SACOG,NaN,5924,214,True


In [4]:
#making sure the locode and agency updated in the data.
## if updated, there will only be one agency listed.
## if multiple agencies listed, go back to add_locode.ipynb and run that notebook

df>>filter(_.locode=='6000')>>count(_.agency)

,agency,n
0,San Francisco Bay Area Rapid Transit District,41


## How many unique prefix codes per agency

### Top Agencies
finding the top agencies with the most unique prefix codes

In [5]:
df >> group_by(_.agency) >> summarize(n=_.prefix.nunique()) >> arrange(-_.n)>>filter(_.n>=15)

,agency,n
291,Los Angeles,39
74,Caltrans,27
292,Los Angeles County,27
459,San Bernardino Associated Governments,24
466,San Diego,22
439,Riverside County,21
492,San Luis Obispo County,21
569,Stockton,21
156,El Dorado County,20
337,Monterey County,19


In [6]:
top = df >> group_by(_.agency) >> summarize(n=_.prefix.nunique()) >> arrange(-_.n)>>filter(_.n>=20)

In [7]:
top

,agency,n
291,Los Angeles,39
74,Caltrans,27
292,Los Angeles County,27
459,San Bernardino Associated Governments,24
466,San Diego,22
439,Riverside County,21
492,San Luis Obispo County,21
569,Stockton,21
156,El Dorado County,20


In [8]:
@interact
def find_agencies_top(agency=top.agency.unique().tolist()):

            count_top = (df>>filter(_.agency==agency)>> count(_.prefix)>>arrange(-_.n))
            
            display(Markdown(f"**Top Prefix Codes {agency} uses:**"))
            display(count_top)


interactive(children=(Dropdown(description='agency', options=('Los Angeles', 'Caltrans', 'Los Angeles County',…

#### Printing top 5 programs for the top agencies


In [9]:
top

,agency,n
291,Los Angeles,39
74,Caltrans,27
292,Los Angeles County,27
459,San Bernardino Associated Governments,24
466,San Diego,22
439,Riverside County,21
492,San Luis Obispo County,21
569,Stockton,21
156,El Dorado County,20


In [12]:
def find_agencies_top(agency=top.agency.unique().tolist()):
    
    for agency in top.agency.unique().tolist(): 
            count_top = ((df>>filter(_.agency==agency)>> count(_.prefix)>>arrange(-_.n)).head(5)>>select(_.prefix))
            
            display(Markdown(f"**Top Prefix Codes {agency} uses:**"))
            display(count_top)
            
    return

In [13]:
agency=top.agency.unique().tolist()
find_agencies_top(agency)

**Top Prefix Codes Los Angeles uses:**

,prefix
27,STP
22,HSIP
12,CML
2,ATP
5,BHLS


**Top Prefix Codes Caltrans uses:**

,prefix
25,STPLN
1,CMLN
10,HP21L
23,STP
0,CML


**Top Prefix Codes Los Angeles County uses:**

...

,prefix
7,BRLO
16,HSIP
5,BHLS
8,BRLS
6,BPMPL


**Top Prefix Codes Stockton uses:**

,prefix
10,HSIP
6,CML
16,STP
1,ATPCML
2,ATPLNI


**Top Prefix Codes El Dorado County uses:**

,prefix
5,BRLO
7,CML
14,HSIP
11,ER
18,STP


Popular codes in top agencies:
   * STP/STLN: Surface Transportation Program/Surface Transportation Program Local National Highway System
   * HSIP: Highway Safety Improvement Program 
   * CML: Congestion Mitigation & Air Quality Improvement Program 
   * BHLS: ?
   * BRLO: Highway Bridge Program (HBP) - Off Federal-aid System 

### Bottom Agencies

In [14]:
(df 
     >> group_by(_.agency) 
     >> summarize(n=_.prefix.nunique())
     >> filter(_.n<=1)
 
)

,agency,n
4,Alameda Corridor Transportation Authority,1
8,Alameda County Transportation Improvement Auth...,1
9,Alameda County Waste Management Authority,1
14,Amador City,1
16,Amador County Transportation Commission,1
...,...,...
638,Westminister,1
644,Willits,1
645,Willows,1
655,Yountville,1


In [15]:
df >> group_by(_.agency) >> summarize(n=_.prefix.nunique()) >> arrange(_.n)>>filter(_.n<=3)

,agency,n
4,Alameda Corridor Transportation Authority,1
8,Alameda County Transportation Improvement Auth...,1
9,Alameda County Waste Management Authority,1
14,Amador City,1
16,Amador County Transportation Commission,1
...,...,...
608,Ukiah,3
628,Walnut Creek,3
633,West Covina,3
639,Westminster,3


In [16]:
bottom = df >> group_by(_.agency) >> summarize(n=_.prefix.nunique()) >> arrange(_.n)>>filter(_.n<=3)

In [17]:
bottom.sample(9)

,agency,n
119,Coronado,1
87,Cerritos,2
527,Scotts Valley,1
57,Buena Park,1
73,Calistoga,1
200,Gridley,2
52,Blythe,2
595,Trinidad,1
653,Yorba Linda,2


In [18]:
agency=bottom.agency.unique().tolist()


def find_agencies_bottom(agency=bottom.agency.unique().tolist()):
    
    for agency in bottom.agency.unique().tolist(): 
            count_bottom = ((df>>filter(_.agency==agency)>> count(_.prefix)>>arrange(_.n)).head(5)>>select(_.prefix))
            
            display(Markdown(f"**Top Prefix Codes {agency} uses:**"))
            display(count_bottom)
            
    return

In [19]:
print(len(agency))

339


In [20]:
find_agencies_bottom(agency)

**Top Prefix Codes Alameda Corridor Transportation Authority uses:**

,prefix
0,HPLUL


**Top Prefix Codes Alameda County Transportation Improvement Authority uses:**

,prefix
0,STP


**Top Prefix Codes Alameda County Waste Management Authority uses:**

...

,prefix
0,HPLUL
1,HSIP
2,STP


**Top Prefix Codes Westminster uses:**

,prefix
2,STP
0,ATP
1,HSIP


**Top Prefix Codes Winters uses:**

,prefix
1,HPLUL
2,STP
0,BRLS


Popular codes in bottom agencies: 
   * FTACML
   * STP/RPSTP
   * HSIP
   * CML 
   * BRLS
   * BRLO
   * CMLNI
    

* 140 agencies with 1 unique prefix code.
* 339 agencies with 3 or less prefix codes. 

## In which Caltrans Districts do these agencies have the most obligations?

### For the agencies in districts that applied the least

* joining the bottom agencies to the full df
* grouping by dsitrict to count the number of obligation occurances 

In [37]:
bottom = (df 
     >> group_by(_.agency) 
     >> summarize(n=_.prefix.nunique())
     >> filter(_.n<=3)
)

In [38]:
bottom_join = inner_join(df, bottom, on = "agency")

In [39]:
bottom_dist = bottom_join >> group_by(_.dist) >> count(_.dist) >> arrange(-_.n)

In [40]:
bottom_dist.rename(columns={'n': 'bottom_n'}, inplace=True)

In [41]:
bottom_dist

,dist,bottom_n
6,7,409
3,4,392
9,10,376
5,6,264
2,3,261
7,8,209
10,11,129
11,12,114
4,5,86
8,9,74


### For the agencies in districts that applied the most

* joining the top agencies to the full df
* grouping by dsitrict to count the number of obligation occurances 

In [42]:
top_join = inner_join(df, top, on = "agency")

In [43]:
top_dist = top_join >> group_by(_.dist) >> count(_.dist) >> arrange(-_.n)

In [44]:
top_dist.rename(columns={'n': 'top_n'}, inplace=True)

In [45]:
top_dist

,dist,top_n
7,7,960
3,3,312
10,10,262
8,8,173
5,5,133
11,11,125
12,12,13
4,4,10
0,0,4
6,6,3


### Number of Obligations for Top and Bottom on District
 
* joining the dataframes we just made to compare the top and bottom easier
* finding which districts have the most obligations 

In [46]:
dist = full_join(bottom_dist, top_dist, on = "dist")

In [47]:
dist>>arrange(_.dist)

,dist,bottom_n,top_n
12,0,NaN,4
10,1,61.0,2
11,2,52.0,1
4,3,261.0,312
1,4,392.0,10
8,5,86.0,133
3,6,264.0,3
0,7,409.0,960
5,8,209.0,173
9,9,74.0,1


In [48]:
dist.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13 entries, 0 to 12
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   dist      13 non-null     int64  
 1   bottom_n  12 non-null     float64
 2   top_n     13 non-null     int64  
dtypes: float64(1), int64(2)
memory usage: 416.0 bytes


In [49]:
dist['bottom_n'] = dist['bottom_n'].fillna(0)

In [50]:
dist['bottom_n'] = dist['bottom_n'].astype(int)

In [51]:
dist.rename(columns={'dist': 'district'}, inplace=True)

In [52]:
dist>>arrange(_.district)

,district,bottom_n,top_n
12,0,0,4
10,1,61,2
11,2,52,1
4,3,261,312
1,4,392,10
8,5,86,133
3,6,264,3
0,7,409,960
5,8,209,173
9,9,74,1


## Findings:



* **339 agencies** have **3 or less** unique prefixes. 
* **80 agencies** have **10 or more** unique prefixes, with the following **9 agenceies** listed with **20 or more**. 
* **District 7 and District 2** have the most obligations out of the top and bottom agencies 



| Top Agencies  | Bottom Agencies (sample of 9)|
| ------------- | ------------- | 
| Los Angeles | Sonoma County Transportation Authority | 
| Caltrans  |Port Of Long Beach |   
| Los Angeles County |Newport Beach |    
| San Bernardino Associated Governments | Portola | 
| San Diego | Calaveras Council Of Governments | 
| Riverside County | Hawthorne | 
| San Luis Obispo County | Corcoran |  
| Stockton | Claremont | 
| El Dorado County | El Cajon |  


